# Loop 2 LB Feedback Analysis

## Key Findings:
- LB score 70.627582 confirmed (CV-LB gap: 0.0000)
- Better validated submission exists at 70.626088
- Target: 68.892432 | Gap: 1.73 points (2.5%)

## Strategy:
1. Find best validated submission
2. Ensemble best N values from multiple validated snapshots
3. Test bbox3 optimizer

In [ ]:
import pandas as pd
import numpy as np
import json
import os
import math
from numba import njit
from glob import glob

# Tree geometry
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

@njit
def score_group(xs, ys, degs, tx, ty):
    n = xs.size
    V = tx.size
    mnx = mny = 1e300
    mxx = mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c, s = math.cos(r), math.sin(r)
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xs[i]
            Y = s * tx[j] + c * ty[j] + ys[i]
            mnx, mxx = min(mnx, X), max(mxx, X)
            mny, mxy = min(mny, Y), max(mxy, Y)
    side = max(mxx - mnx, mxy - mny)
    return side * side / n

print('Scoring functions ready')

In [ ]:
def get_n_scores(df):
    """Get score for each N value"""
    scores = {}
    for n in range(1, 201):
        mask = df['id'].str.startswith(f'{n:03d}_')
        group = df[mask]
        if len(group) != n:
            continue
        xs = group['x'].str[1:].astype(float).values
        ys = group['y'].str[1:].astype(float).values
        degs = group['deg'].str[1:].astype(float).values
        scores[n] = score_group(xs, ys, degs, TX, TY)
    return scores

def total_score(scores_dict):
    return sum(scores_dict.values())

print('Helper functions ready')

In [ ]:
# Find all validated snapshots with LB scores
snapshot_base = '/home/nonroot/snapshots/santa-2025'
validated_snapshots = []

for snap_dir in sorted(os.listdir(snapshot_base)):
    session_path = os.path.join(snapshot_base, snap_dir, 'code', 'session_state.json')
    submission_path = os.path.join(snapshot_base, snap_dir, 'code', 'submission.csv')
    
    if os.path.exists(session_path) and os.path.exists(submission_path):
        try:
            with open(session_path) as f:
                state = json.load(f)
            
            # Find successful submissions
            for sub in state.get('submissions', []):
                lb = sub.get('lb_score')
                if lb and lb != '' and not sub.get('error'):
                    validated_snapshots.append({
                        'dir': snap_dir,
                        'lb_score': float(lb),
                        'submission_path': submission_path
                    })
                    break
        except:
            pass

print(f'Found {len(validated_snapshots)} validated snapshots')
validated_snapshots = sorted(validated_snapshots, key=lambda x: x['lb_score'])
print('\nTop 10 validated snapshots:')
for v in validated_snapshots[:10]:
    print(f"  LB: {v['lb_score']:.12f} - {v['dir']}")

In [ ]:
# Load the best validated submission
best_snap = validated_snapshots[0]
print(f"Best validated: {best_snap['lb_score']:.12f}")

df_best = pd.read_csv(best_snap['submission_path'])
scores_best = get_n_scores(df_best)
print(f"Total score: {total_score(scores_best):.6f}")
print(f"\nTop 10 N values by contribution:")
for n, s in sorted(scores_best.items(), key=lambda x: -x[1])[:10]:
    print(f"  N={n:3d}: {s:.6f}")

In [ ]:
# Try to ensemble best N values from multiple validated snapshots
# Load top 5 validated submissions
top_submissions = []
for v in validated_snapshots[:5]:
    df = pd.read_csv(v['submission_path'])
    scores = get_n_scores(df)
    top_submissions.append({
        'dir': v['dir'],
        'lb_score': v['lb_score'],
        'df': df,
        'scores': scores
    })
    print(f"Loaded {v['dir']}: total={total_score(scores):.6f}")

print(f"\nLoaded {len(top_submissions)} submissions for ensemble")

In [ ]:
# Find best N from each submission
best_n_scores = {}
best_n_source = {}

for n in range(1, 201):
    best_score = float('inf')
    best_src = None
    
    for sub in top_submissions:
        if n in sub['scores'] and sub['scores'][n] < best_score:
            best_score = sub['scores'][n]
            best_src = sub['dir']
    
    if best_src:
        best_n_scores[n] = best_score
        best_n_source[n] = best_src

ensemble_total = sum(best_n_scores.values())
print(f"\nEnsemble total score: {ensemble_total:.6f}")
print(f"Best single submission: {top_submissions[0]['lb_score']:.6f}")
print(f"Improvement: {top_submissions[0]['lb_score'] - ensemble_total:.6f}")

# Count how many N values come from each source
from collections import Counter
source_counts = Counter(best_n_source.values())
print(f"\nN values per source:")
for src, count in source_counts.most_common():
    print(f"  {src}: {count} N values")

In [ ]:
# Check if ensemble is actually different from best single
if ensemble_total < top_submissions[0]['lb_score'] - 1e-6:
    print("Ensemble is better! Creating ensemble submission...")
    
    # Build ensemble submission
    ensemble_rows = []
    for n in range(1, 201):
        best_src = best_n_source[n]
        for sub in top_submissions:
            if sub['dir'] == best_src:
                mask = sub['df']['id'].str.startswith(f'{n:03d}_')
                group = sub['df'][mask]
                for _, row in group.iterrows():
                    ensemble_rows.append(row)
                break
    
    ensemble_df = pd.DataFrame(ensemble_rows)
    
    # Verify score
    ensemble_scores = get_n_scores(ensemble_df)
    print(f"Ensemble verified score: {total_score(ensemble_scores):.6f}")
else:
    print("Ensemble is NOT better than best single submission")

In [ ]:
# Check if bbox3 is available and test it
import subprocess
import shutil

# Copy bbox3 from snapshot
bbox3_src = '/home/nonroot/snapshots/santa-2025/21198927060/code/bbox3'
bbox3_dst = '/home/code/bbox3'

if os.path.exists(bbox3_src):
    shutil.copy(bbox3_src, bbox3_dst)
    os.chmod(bbox3_dst, 0o755)
    print(f"Copied bbox3 to {bbox3_dst}")
    
    # Test bbox3
    result = subprocess.run([bbox3_dst, '-h'], capture_output=True, text=True, timeout=5)
    print(f"\nbbox3 help:")
    print(result.stdout[:500] if result.stdout else result.stderr[:500])
else:
    print(f"bbox3 not found at {bbox3_src}")

In [ ]:
# Copy best validated submission to /home/submission
import shutil
os.makedirs('/home/submission', exist_ok=True)

# Use the best validated submission (70.626088)
best_submission_path = validated_snapshots[0]['submission_path']
shutil.copy(best_submission_path, '/home/submission/submission.csv')

print(f"Copied best validated submission to /home/submission/submission.csv")
print(f"Expected LB score: {validated_snapshots[0]['lb_score']:.12f}")

# Verify
df_verify = pd.read_csv('/home/submission/submission.csv')
scores_verify = get_n_scores(df_verify)
print(f"Verified score: {total_score(scores_verify):.6f}")